<a href="https://colab.research.google.com/github/tractorjuice/Building_Wardley_BoK/blob/main/Demo_Custom_Tools_for_Agents_(Shared).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [ ]:
!pip -q install langchain openai

### Enter your OpenAI API Key in this Section

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

### Import Dependencies

In [ ]:
from langchain.agents import load_tools
from langchain.utilities import TextRequestsWrapper
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

In [ ]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.llms import OpenAI
import requests

In [ ]:
llm = OpenAI(temperature=0)

### TOOLS

### Build the Tools by Subclassing the BaseTool class

In [ ]:
def get_police_forces():
    url = "https://data.police.uk/api/forces"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return None

def search_police_forces(query):
    police_forces = get_police_forces()

    if not police_forces:
        print("Error: Unable to fetch police forces")
        return

    matching_forces = []

    for force in police_forces:
        if query.lower() in force["name"].lower():
            matching_forces.append(force)

    return matching_forces

class search_police_force(BaseTool):
    name = "search_police_force"
    description = "useful for when you need to answer questions about police forces, such as the id	Unique force identifier and Police Force name"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return search_police_forces(query)

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("API does not support async")

# Example usage
force_id = "devon"
police_force_data = search_police_forces(force_id)
print (police_force_data)

In [ ]:
def get_police_force_id(force_id):
    url = f"https://data.police.uk/api/forces/{force_id}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return None


class get_police_force_details(BaseTool):
    name = "get_police_force_details"
    description = "useful for when you need to answer questions about a specific police force such as the website, ways to keep informed, telephone number, Unique force identifier or Force name"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return get_police_force_id(query)

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("API does not support async")

# Example usage
force_id = "devon-and-cornwall"
police_force_data = get_police_force_id(force_id)
print (police_force_data)

In [ ]:
def get_police_force_people(force_id):
    url = f"https://data.police.uk/api/forces/{force_id}/people"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return None


class get_police_force_peoples(BaseTool):
    name = "get_police_force_peoples"
    description = "useful for when you need to answer questions about Senior officer bio"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return get_police_force_people(query)

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("API does not support async")

# Example usage
force_id = "leicestershire"
police_force_data = get_police_force_people(force_id)
print (police_force_data)

In [ ]:
def get_neighbourhoods(force_id):
    url = f"https://data.police.uk/api/{force_id}/neighbourhoods"
    cleaned_url = url.replace("'", "")
    response = requests.get(cleaned_url)
    neighbourhoods = response.json()

    if response.status_code == 200:
        names = [entry['name'] for entry in neighbourhoods]
        names_string = ', '.join(names)
        return (names_string)
    else:
        return None

class get_police_neighbourhoods(BaseTool):
    name = "get_police_neighbourhoods"
    description = "useful for when you need to answer questions about the name for the police force neighbourhood. You will need to use GetPoliceforce first to get the force name for the API"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return get_neighbourhoods(query)

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("API does not support async")

# Example usage
force_id = "\'devon-and-cornwall\'"
neighbourhood_data = get_neighbourhoods(force_id)
print (neighbourhood_data)

In [ ]:
def locate_neighbourhood(latlong):

    url = f"https://data.police.uk/api/locate-neighbourhood?q={latlong}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        return None

class locate_police_neighbourhood(BaseTool):
    name = "locate_police_neighbourhood"
    description = "useful for when you need to answer questions about the location of a police neighbourhood using lattitude, longitude numbers. Pass lattitude and longitude as a string"

    def _run(self, query: str) -> str:
        """Use the tool."""
        return locate_neighbourhood(query)

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("API does not support async")

# Example usage
latlong = "50.3656 ,-4.1423"
neighbourhood_data = locate_neighbourhood(latlong)
print (neighbourhood_data)

### Create Tools Config

In [ ]:
# Load the tool configs that are needed.
tools = [
    Tool(
        name="SearchPoliceForce",
        func=search_police_force.run,
        description="useful for when you need to answer questions about police forces, such as the id	Unique force identifier and Police Force name"
    ),
    Tool(
        name="GetPoliceforceDetails",
        func=get_police_force_details.run,
        description="useful for when you need to answer questions about a specific police force such as the website, ways to keep informed, telephone number, Unique force identifier or Force name"
    ),
    Tool(
        name="GetPoliceforcePeople",
        func=get_police_force_peoples.run,
        description="useful for when you need to answer questions about Senior officer bio"
    ),
        Tool(
        name="GetPoliceforceNeighbourhood",
        func=get_police_neighbourhoods.run,
        description="useful for when you need to answer questions about the name for the police force neighbourhood. You will need to use GetPoliceforceDetails first to get the force name for the API"
    ),
    Tool(
        name="LocatePoliceNeighbourhood",
        func=locate_police_neighbourhood.run,
        description="useful for when you need to answer questions about the location of a police neighbourhood using lattitude, longitude numbers. Pass lattitude and longitude as a string"
    )
]

In [ ]:
tools = [search_police_force(), get_police_force_details(), get_police_neighbourhoods(), locate_police_neighbourhood()]

In [ ]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

### Use Agent to Ask Questions - Agent will pick best tool for the job

In [ ]:
# Locate police force by lat, long [Still Work in progress]
agent.run("Get the police chief for latitude 50.3656, longitude -4.1423")

In [ ]:
agent.run("Get the police force neighbourhoods for devon")

In [ ]:
agent.run("Get get the senior police officer bios at leicestershire police force")